# MCMC para generación de texto coherente

## Índice:
* [Introducción](#intro)
* [Metodología](#meth)
* [Implementación y resultados](#imp-res)
    * [Limpieza de Texto](#limp)
    * [Matriz de Transición](#pmatriz)
    * [Implementación del algoritmo Metropolis-Hasting](#mh)
* [Conclusiones](#conc)
* [Bibliografía](#ref)
* [Anexos](#anex)
    * [Código adicional](#code)



## Introducción <a class="anchor" id="intro"></a>

Día con día, diversas aplicaciones nos ayudan a producir texto, ya sea Word ayudándonos a crear un documento sin errores gramaticales u ortográficos, Google dándonos resúmenes cortos y rápidos de páginas webs o hasta Siri que se comunica con nosotros de forma particularmente humana, pero ¿alguna vez te has preguntado cómo es que una computadora encargada de "dar vida" a estas aplicaciones, que únicamente sabe pensar en ceros y unos, puede interpretar y producir lenguaje humano? El Procesamiento del Lenguaje Natural (NLP) es un campo de estudio de la inteligencia artificial que se enfoca en la interacción entre las computadoras y el lenguaje humano. Éste utiliza algoritmos y técnicas para comprender, interpretar y generar texto (Jurafsky & Martin, 2020). Una de las herramientas más utilizadas en el NLP son las cadenas de Márkov, que son modelos matemáticos que representan la probabilidad de transición entre diferentes estados de un proceso estocástico.

Si quisieramos ir aún más lejos, podríamos implementar herramientas un poco más poderosas como el MCMC. El Muestreo de Markov Chain Monte Carlo (MCMC) es una técnica de muestreo ampliamente utilizada en estadística y aprendizaje automático para estimar distribuciones de probabilidad y simular sistemas complejos (Robert & Casella, 2013). El objetivo de este proyecto es simular un texto que imite el estilo de escritura de un autor a elección del usuario. Pero esto puede ser volverse bastante complicado pues cada autor tiene su propia forma de escribir y expresarse, es decir, cada quién tiene sus vicios del lenguaje, tono, etc. De igual manera, no es lo mismo reproducir el texto plano de un libro que el un tweet, ya que cada uno tiene sus dificultades de procesamiento. Mientras el texto de un libro puede llegar a ser complejo por las herramientas literarias que puede llegar a utilizar, un tweet igual puede mostrar dificultades por los recursos que utilizan como las menciones, hashtags, emojis, etc.

En este reporte, describimos la implementación de un algoritmo de MCMC llamado Metropolis-Hastings para generar texto coherente a partir de un texto dado (o palabra). La generación de texto coherente es un problema relativamente dificil en el procesamiento del lenguaje natural y el aprendizaje automático, y MCMC ofrece una forma sistemática y flexible de explorar diferentes combinaciones de palabras para generar texto significativo (Andrieu et al., 2003).


## Metodología <a class="anchor" id="meth"></a>

El método MCMC (Markov Chain Monte Carlo) es una técnica de simulación numérica que se utiliza para estimar la distribución de probabilidad de una variable aleatoria (Robert & Casella, 2013). La idea detrás del método es construir una cadena de Markov que tenga como distribución estacionaria la distribución de probabilidad que se quiere estimar.

Supongamos que nosotros queremos crear un texto aleatorio parecido de un autor $X$. Cada palabra que se genere dentro del texto, seguirá una distribución teórica $\pi$, la cual deseamos simular para generar el texto. ¿Cómo podríamos construir una cadena ergódica que admita a $\pi$ como distribución invariante? 

Dado un texto, podemos crear una matriz de transición o estados $P$ que nos indique cuál es la probabilidad de pasar de una palabra dada a otra. Para ello proponemos el siguiente pseudocódigo:

- Tokenizamos el texto en una lista
- Recorremos cada palabra de la lista
- Para cada palabra realizamos lo siguiente:
    - Guardamos la siguiente palabra (ya sea en un array o diccionario) junto con un 1 que servirá de contador
    - Si ya se había guardado esa palabra antes, le sumamos 1

![Diagram0](RenderImages/Diagram0.png)

Al tener la matriz de transición y suponiendo que el texto es una cadena ergódica, entonces por teorema tenemos que, sean $i$ y $j$ palabras del texto:

$$\pi(j)=\lim_{n \to \infty}p_{ij}^{(n)}$$

Y por teorema ergódico, podemos estimar esperanzas de funciones utilizando promedios aritméticos y así simular la mejor opción para la siguiente palabra en el texto. 

Dado todo lo anterior, para generar un texto después de una palabra dada, primero podríamos proponer los candidatos que muestra la matriz $P$ y escoger alguno dada su distribución propuesta $\pi_0$. 
Para que el texto  converja a lo que queremos, es decir que acepte a $\pi$ como distribución invariante, calculamos un coeficiente o probabilidad de aceptación para ver si nos quedamos con la palabra propuesta o no, lo cual nos creará una nueva probabilidad de transición. Esto se simulará para un número grande de iteraciones, ya que por TCL, entre más iteraciones se tengan, más preciso y de mejor calidad será el texto que se genere, pero obviamente tomará más tiempo.

Para ello, proponemos el siguiente pseudocódigo para la generación de texto:

![Diagram2](RenderImages/Diagram2.png)

## Implementación y resultados <a class="anchor" id="imp-res"></a>

### Limpieza de Texto <a class="anchor" id="limp"></a>

Antes de implementar el algoritmo hay varias cosas que se deben tomar en cuenta. Primero, debemos pensar si queremos filtrar el texto antes de aplicar el algoritmo, ya que los signos de puntuación, caracteres especiales (no ASCII) y hasta las letras mayúsculas pueden alterar la forma en la que se genera el texto.

![Diagram1](RenderImages/Diagram1.png)

Al escoger no filtrar nuestro texto, corremos el riesgo que nuestro texto no sea tan aleatorio como quisiéramos y que no sea del todo lógico o coherente. Por el otro lado, filtrarlo haría que la convergencia sea más rápida, creando un texto más conciso en el estilo deseado, pero sin ningún sentido lógico gramatical; por ejemplo, en español los acentos pueden hace que todo el sentido de una palabra sea completamente distinto.

Esta misma cuestión sobre el filtrado me trajo una duda distinta: ¿qué sucedería si quisiéramos generar tweets? El problema de querer filtrar texto en un tweet es que al eliminar caracteres especiales como hashtags y menciones corremos el riesgo de romper completamente con la lógica del tweet.

![Hashtags](RenderImages/tweet.png)

Por eso mismo se implementó el algoritmo no únicamente con textos en español, sino también en inglés, aplicando en ambos casos un caso donde se ha filtrado el texto y uno en el que no. En el caso de los tweets, decidimos no filtrar nada para ver qué sucedía. Los textos utilizados para la implementación fueron “Don Quijote de la Mancha” y “El retrato de Dorian Gray” ambos en formato de texto. Para los tweets se descargó una base de datos en formato csv de la página [TTA - Search](https://www.thetrumparchive.com/). Los códigos para la lectura de los archivos y la limpieza del texto se encuentran en el apartado de Código Adicional en Anexos y lo que hacen es leer los distintos archivos, ya sea limpiarlos o no, y crear una lista con las palabras los textos para que se puedan procesar luego.

### Matriz de Transición <a class="anchor" id="pmatriz"></a>

Para implementar MCMC en la generación de texto, primero construimos una matriz de transición que describe las probabilidades de transición entre palabras consecutivas en un texto. Para ello construimos una función que toma una lista de palabras y crea un diccionario anidado que representa la matriz de transición. Cada palabra en el texto se utiliza como una llave en el diccionario, y sus valores son otro diccionario que contiene las palabras que siguen a la palabra clave y la frecuencia relativa con la que aparecen. Esta matriz de transición se utiliza para guiar la generación de texto en el algoritmo de MCMC.

In [ ]:
# Función que recibe un texto en forma de lista y te calcula la frecuencia con la que aparecen ciertas palabras
# despues de una palabra específica
def probabilidadesTransición(words):
# P será nuestro diccionario que tendrá como llave todas las palabras disponibles en el texto
# y como valor OTRO DICCIONARIO, cuyo llave volverán a ser todas las palabras disponibles en el texto
# y como valor la probabilidad que despues de la primera palabra (primera llave) siga la segunda palabra
# (segunda llave)
    P = {}
#Recorremos cada palabra en la lista y analizamos la primera palabra y su sucesora
    for i in range(len(words)-1):
#X: presente, Y: futuro
        X = words[i]
        Y = words[i+1]
#Si x no está en el diccionario, la agregamos y agregamos como valor otro diccionario incluyendo como llave a Y y agregarle
# 1 a la frecuencia en la que aparece Y despues de X
        if P.get(X) is None:
            P[X] = {}
            P[X][Y] = 1
#Si X ya está, ahora checamos si Y está como llave en el diccionario en el valor de X
        else:
            if P[X].get(Y) is None:
#Si no está ponemos como valor 1 y si sí está le agregamos 1 a la frecuencia
                P[X][Y] = 1
            else:
                P[X][Y] += 1
#Teniendo ya el diccionario con las frecuencias, ahora toca sacar las probabilidades de que salga cada palabra
#Recorremos cada llave del diccionario y sumamos todos los valores encontrados en el diccionario de esa llave
    for i in P.keys():
        s = float(sum(P[i].values()))
#Ahora trecorremos cada llave del segundo diccionario (del valor de la primera llave) y ajustamos el valor de la frecuencia
#dividiendola entre la suma anterior y asi obteniendo una probabilidad elemento del (0,1)
        for k in P[i].keys():
            P[i][k] = P[i][k]/s
#Devolvemos la matriz de transición
    return P

### Implementación del algoritmo Metropolis-Hasting <a class="anchor" id="mh"></a>

Para implementar el algoritmo, crearemos pimero una función alpha que calcule la propapilidad de aceptación de la nueva palabra y que penalice las transiciones poco probables, calculando el negativo del logaritmo de la probabilidad de transición, ya que son probabilidades, es decir $p\in (0,1)$. Esta función, recibirá la palabra actual, la propuesta siguiente y la matriz de transición, devolviendo así una propabilidad de aceptación.

In [ ]:
def alpha(P, current_word, next_word):

#Este filtro es importante en este caso que no filtramos el texto (En el caso de no filtrar el texto no es neceario el if), ya que sucedía
#muchas veces que la función estaba tratando de acceder a una llave existe en el diccionario o palabra en la matriz de probabilidades. 
#Esto sucedía por palabras que incluían un come o algun signo de admiración o exclamación.

#Para corregir este error, se agregó una verificación en la función para asegurarnos de que solo intente acceder a claves existentes 
#en el diccionario P. Si la clave no existe, se devuelve un valor muy alto para que esa transición sea poco probable.
    if current_word not in P or next_word not in P[current_word]:
        return float('inf')
    return -math.log(P[current_word][next_word])

Primero muestrearemos una palabra de la distribución de probabilidad obtenida (la matriz o diccionario creado en la función anterior). Despues implementamos el algoritmo de Metropolis-Hastings para generar texto utilizando la matriz de transición $P$ y la propabilidad de aceptación $\alpha$. La función toma como entrada una palabra inicial (seed_word), la longitud deseada del texto generado y el número de iteraciones para realizar en cada paso del algoritmo (entre más iteraciones, más preciso) y regresa una string variable que será nuestro texto.  Al utilizar la probabilidad de aceptación y la alpha asociada con las palabras, el algoritmo favorece las transiciones de palabras con mayor probabilidad, lo que puede resultar en un texto generado más coherente.

In [ ]:
def generate_text_mh(P, seed_word, length, iteraciones):

#Creamos una lista donde almacenaremos las palabras que contendrá el texto.
#La variable current_word cambiará conforme vayamos agregando palabras al texto, empezando por la semilla
    current_word = seed_word
    text = [current_word]

#Recorremos el largo que queremos que sea el texto
    for i in range(length):

#Vemos si la palabra se encuentra en el diccionario, es decir, si el autor utilizaría esa palabra en alguno de sus textos
#Si no es el caso, tronamos la función
        if P.get(current_word) is None:
            print("Saavedra jamás diría eso")
            break

#Si la palabra sí está en el diccionario (matriz), comenzamos el algoritmo Metropolis-Hasting para muestrear la siguiente palabra en la cadena
#Esto lo haremos el numero de iteraciones que se desee
        for i in range(iteraciones):

#Dada la palabra en la que nos encontremos, sacaremos del diccionario las palabras que le pueden seguir como una lista, y la
#probabilidad como otra
#Recordemos que la matriz de transiciones es un diccionario de diccionarios, entonces al buscar la palabra actual current_word
#en el diccionario, nos mostrará otro diccionario con las palabras que le pueden seguir como llaves y como valores las probabilidades
            next_word_candidates = list(P[current_word].keys())
            next_word_probabilities = list(P[current_word].values())

#Teniendo la lista de palabras candidatas y probabilidades, seleccionamos una palabra utilizando la distribución propuesta 
#En este caso, proponemos distribución del modelo que sacamos anterirmente, es decir, utilizando la matriz de trancisiones (diccionario)
#También podríamos proponer que la distribución fuera uniforme para todas las palabras, pero tendría un tiempo de convergencia menor.
            proposed_next_word = random.choices(next_word_candidates, weights=next_word_probabilities, k=1)[0]

#Calculamos la razón de aceptación con la función alpha
# "Lanzamos la moneda" y vemos si nos quedamos con la palabra o nos movemos a otra

# Probabilidad de no cambiar
            a = alpha(P, current_word, text[-1])
# Probabilidad de cambio a siguiente palabra
            a2 = alpha(P, current_word, proposed_next_word)
#Calculamos la propabilidad de aceptación
            probabilidad_aceptacion = min(1, math.exp(a - a2))

# Vemos si aceptamos o rechazamos la palabra propuesta, generando valores de la distribucion Unif(0,1) y viendo si es menor a 
# Mi probabilidad de aceptación
            if random.random() < probabilidad_aceptacion:
#Si es mayor, definimos la nueva palabra
                next_word = proposed_next_word
                break
#Al terminar las iteraciones agregamos la nueva palabra a la lista
        text.append(next_word)
#Cambiamos la variable current_word a la nueva palabra agregada para seguir con el ciclo otra vez
        current_word = next_word

#Terminando el algoritmo, regresamos el texto todo junto
    return ' '.join(text)

### Resultados

#### Textos en Español

<em>Seed: "amarillo"; Tamaño: 50 </em><br><br>
<strong>Sin Filtrar</strong>: amarillo y sólo os conozco, por hecho con mucho cuando, movido a todos se llamaba Juan Rufo, jurado nada), que estáis tan buena fe se podrían reprensentallas, y los rostros son los desengaños no se contentaron de caballería y temerosas y la historia de rescate o a quien tiene usurpado; que

<strong>Texto Filtrado</strong>: amarillo y mas deseos y diciendo ah traidor que se le parecio y guy de aquella gran reina de llamar el primero de leer de caballerias que estaban la noticia del caballo rocinante albarde el de don quijote el cual ya pasados siglos y aunque la hacienda alli tendido en ningun

#### Textos en Inglés

<em>Seed: "yellow"; Tamaño: 50 </em><br><br>
<strong>Sin Filtrar</strong>: yellow piazza of evil, with myriads of myself," said Dorian. Tell me know their coats and give him the aged, I to put her with dyed hair and you really changed? Or rather, I am afraid of, I am quite obvious. But it was, but that I can it began to

<strong>Texto Filtrado</strong>: yellow chinese box twentyseven i was nothing fearful about it all a burden to death of the women were pictured to see the emotion no you must come some strange conjectures as much better go through long _clarin_ of the room that it is like fire to linger sometimes think certainly

#### Tweet de Donald Trump

<em>Seed: "America"; Tamaño:40 </em><br><br>

"America could extort $1,000,000.00 from the crowd. He is willing to Create Jobs, Jobs, Border than a tax cuts &amp, @gatewaypundit. @jheil at R's. Shame! ....President. We need to establish a country could be winning their best opportunity to a great"

## Conclusiones <a class="anchor" id="conc"></a>

El algoritmo Metropolis-Hastings es bastante bueno generar muestras de distribuciones de probabilidad no triviales desconocidas (como los textos en nuestro caso) mediante la construcción de una cadena de Markov cuya distribución estacionaria coincide con la distribución objetivo. Como pudimos ver, es relativamente fácil de implementar y converge relativamente rápido si se tiene una muestra grande de datos, regalando resultado no tan malos. De igualmanera se puede adaptar a una amplia variedad de problemas.

Sin embargo, el algoritmo Metropolis-Hastings también tiene algunas desventajas. En primer lugar, puede ser ineficiente en ciertos casos, ya que la convergencia a la distribución estacionaria puede ser lenta. La elección de una distribución propuesta adecuada es crucial para la eficiencia del algoritmo, pero esto puede ser difícil en la práctica. 

Algunas formas en las que se mejoraría el MCMC para la generación de texto pueden ser las siguientes:
1. Utilizar un modelo de lenguaje más avanzado, como un modelo de n-gramas o un modelo neuronal, para obtener mejores estimaciones de las probabilidades de transición entre palabras.

2. Mejorar la generación de texto utilizando modelos más avanzados, como redes neuronales recurrentes (RNN).

## Referencias <a class="anchor" id="ref"></a>

##### Repositorio del proyecto
* https://github.com/Buebito/MCMC-Text-Generation.git
##### Bibliografía
* Andrieu, C., de Freitas, N., Doucet, A. et al. An Introduction to MCMC for Machine Learning. Machine Learning 50, 5–43 (2003). https://doi.org/10.1023/A:1020281327116
* Gerlach, M. and Font-Clos, F. (2020) “A standardized project gutenberg corpus for statistical analysis of Natural Language and Quantitative Linguistics,” Entropy, 22(1), p. 126. Available at: https://doi.org/10.3390/e22010126. 
* Jurafsky, D., & Martin, J. H. (2020). Speech and Language Processing. An Introduction to Natural Language Processing, Computational Linguistics, and Speech Recognition (3rd edition).
* Robert, C.; Casella, G. (2010). Introducing Monte Carlo Methods with R. Berlin: Springer-Verlag.
* (2018). Text generation using Monte-Carlo Sampling.

##### Archivos externos
* Brendan (2016) “The Trump Archive.” Available at: https://www.thetrumparchive.com. 
* Jesús  Darío (2017) El quijote en Texto Plano, El Quijote. GitHub. Available at: https://gist.github.com/jsdario/6d6c69398cb0c73111e49f1218960f79 (Accessed: April 21, 2023). 

## Anexos <a class="anchor" id="anex"></a>

El Jupyter Notebook completo del proyecto con todo el código y los resultados obtenidos, así como este reporte, los archivos txt y la base de datos csv se pueden encontrar en el repositorio de github [MCMC-Text-Generation](https://github.com/Buebito/MCMC-Text-Generation.git), el cual se seguirá actualizando.

### Código adicional <a class="anchor" id="code"></a>

#### Paqueterías necesarias

In [ ]:
import random
import math
import pandas as pd
#Paqueterías de filtrado
from nltk.corpus import stopwords
from unidecode import unidecode
import re
from collections.abc import MutableMapping

#### Lectura de texto y Tokenizado en lista de palabras

##### Sin limpieza de texto

In [ ]:
#Abre el archivo txt y lo convierte en una lista de palabras
with open("El Quijote.txt", "r", encoding="utf-8") as file:
    content = file.read()
    words = content.split()

##### Con limpieza de Texto

In [ ]:
#Recibe una string y devuelve la string sin puntuaciones o signos raros.

def limpieza(text):
#Unidecode toma un objeto de cadena, que posiblemente contenga caracteres no ASCII, y devuelve una cadena que se puede 
#codificar de forma segura en ASCII. En este caso se utilizó para remover acentos y emojis
    text = unidecode(text)
#Minúsculas
    text = text.lower()
#Eliminar signos de interrogación, exclamación y otros
    text = re.sub(r'[^\w\s]', '', text)
#----------------------
    return text

In [ ]:
#Abre el texto y se guarda en string
file = open("El Quijote.txt", "r", encoding="utf-8")
text = file.read()
file.close()
#Lo limpia y lo tokeniza
text=limpieza(text)
words = text.split()

##### Lectura de CVS (como dataframe)

In [ ]:
df = pd.read_csv('tweets_01-08-2021.csv')

#función que transforma una columna de un dataframe en un texto plano
def colToText(df_columna):
    l=list(df_columna)
    return " ".join(l)

#lo convertimos en lista de palabras
text=colToText(df.text)
words=text.split()

##### Ejemplo de cómo implementar las funciones

Teniendo ya la lista de palabras, sea filtrada o no, creamos el modelo de la cadena de Márkov, es decir la matriz de transición, con la función <em>probabilidadesTransición()</em>

In [ ]:
modelo = probabilidadesTransición(words)

Teniendo el modelo o la matriz P ya lista, creamos una variable <em>seed_word</em> que será la primera palabra con la que comenzará el texto. También sería prudente considerar crear una variable <em>length</em>, para el tamaño del texto deseado (tamaño en palabras) e <em>iteraciones</em>, para el numero de iteraciones que se deseen para el Metropolis-Hasting.

Una vez teniendo las variables, las agregamos a la función <em>generate_text_mh<em> para que nos genere el texto deseado.


In [ ]:
seed_word = "amarillo"
generated_text = generate_text_mh(modelo, seed_word, length=50, iteraciones=100000)
print(generated_text)

#### Código de Diagramas de Pseudocódigo

```mermaid
flowchart LR;
    A["current_word"]-->B["next_word"]-->C["Is P[current_word][next_word] in P?"]
    C-->|YES|D["Add P[current_word][next_word] + 1"]
    C-->|NO|E["Set P[current_word][next_word] = 1"]
```

```mermaid
flowchart TD;
    A("current_word")-->B("Propuesta de Palabra")-->C("Calculamos alpha")-->D("Calculamos Probabilidad de Aceptación Y");
    D-->E["Simulamos X-Unif(0,1)"]-->F["Aceptamos si X < Y"]-->B
```

```mermaid
flowchart LR;
    A("ese fue el fin.")-->B["CAPITULO 2"];
    C("ese fue el fin")-->D["de la historia de nuestro querido amigo"];
```